
# **数据挖掘——Home Credit Default Risk**

Authors：李林（3120220938）、杨洋（3220211141）、敬甲男（3220221052）、李翰杰（3120220936）

github地址：https://github.com/leealim/kaggle-Home-Credit-Default-Risk

---


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc

df = pd.read_csv('../data/app_tr.csv')
df_te = pd.read_csv('../data/app_te.csv')


In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
for col in df:
    if df[col].dtype == 'object':
        if len(list(df[col].unique())) <= 2:
            le.fit(df[col])
            df[col] = le.transform(df[col])
            df_te[col] = le.transform(df_te[col])

df = pd.get_dummies(df)
df_te = pd.get_dummies(df_te)

train_labels = df['TARGET']
df, df_te = df.align(df_te, join = 'inner', axis = 1)
df['TARGET'] = train_labels


In [ ]:
import torch


df_te= np.array(df_te)
x_test = df_te[:, 2:]

In [ ]:
from torch.utils.data import DataLoader, Dataset
import torch.utils.data as data

class MyDataset(Dataset):
    def __init__(self, data_root, data_label):
        super(MyDataset, self).__init__()
        self.data = data_root
        self.label = data_label

    def __getitem__(self, index):
        data = self.data[index]
        labels = self.label[index]
        return data, labels

    def __len__(self):
        return len(self.data)


    def get_dataset(x,y):
        dataset = MyDataset(x, y)
        datas = DataLoader(dataset, batch_size=256, shuffle=False, sampler=None, \
                        batch_sampler=None, num_workers=0, collate_fn=None, pin_memory=False, \
                        drop_last=False, timeout=0, worker_init_fn=None, multiprocessing_context=None)
        return datas

dataset = MyDataset.get_dataset(x_train,y_train)
train_size = int(len(dataset)*0.7)
val_size = len(dataset)-train_size
train, val = data.random_split(dataset, [train_size, val_size])
x_train.shape

In [ ]:
import lightning as L
import torch.nn.functional as F
import torch.nn as nn

class MLP(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(nn.Linear(237, 128), nn.ReLU(),
                                     nn.Linear(128, 64), nn.ReLU(),
                                     nn.Linear(64,2))

    def forward(self, x):
        # in lightning, forward defines the prediction/inference actions
        y_pre = self.encoder(x)
        return out

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop. It is independent of forward
        x, y = batch
        y_pre = F.log_softmax(self.encoder(x), dim=1)
        loss = F.nll_loss(y_pre, y)
        self.log("train_loss", loss,on_step=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


autoencoder = MLP()
trainer = L.Trainer(max_epochs=20)
trainer.fit(autoencoder, dataset)








In [ ]:
model = autoencoder.eval().cuda(device=0)
x=torch.from_numpy(x_test[0]).float().unsqueeze(0).cuda(0)
res=model(x).data.cpu().numpy()
print(res)


In [ ]:
res=res[0]
target=[]
for i in res:
    if i[0]<0 | i[1]<0 :
        
    i=i/i.sum()
    target.append(i)
type(target)
target=np.array(target)
type(target)
#生成答案
test = pd.read_csv('../data/app_te.csv')
index=test['SK_ID_CURR']
ans=pd.DataFrame(target[:,1],columns = ['TARGET'])
result=pd.concat([index,ans],axis=1)
result.to_csv('../result/'+version+'/submission_n.csv',encoding = 'utf-8',index = 0)
result.shape
